In [61]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model, metrics, model_selection
import random
import time
import datetime

In [62]:
train_data = pd.read_csv("02012009_29122017.csv")
test_data = pd.read_csv("02012018_31122018.csv")
test_data.head(5)

,Date,Open Price,Close Price,High Price,Low Price,Volume
0,02-Jan-2018,2683.73,2695.81,2695.89,2682.36,1846463232
1,03-Jan-2018,2697.85,2713.06,2714.37,2697.77,2090595328
2,04-Jan-2018,2719.31,2723.99,2729.29,2719.07,2100767744
3,05-Jan-2018,2731.33,2743.15,2743.45,2727.92,1918869120
4,08-Jan-2018,2742.67,2747.71,2748.51,2737.60,1894823936


- 只留最高點和最低點
- 再加上一個'movement'代表當天的走勢
- 平移'movement'對齊今天數據與明天的走勢
- 然後把string的date改成int的timestamp

In [63]:
train_data = train_data.drop(['High Price', 'Low Price', 'Volume'], axis=1)
test_data = test_data.drop(['High Price', 'Low Price', 'Volume'], axis=1)

train_mask = train_data['Open Price']>train_data['Close Price']
test_mask = test_data['Open Price']>test_data['Close Price']

train_data.insert(3,'movement',1)
test_data.insert(3,'movement',1)

train_data.loc[train_mask, 'movement']=0
test_data.loc[test_mask, 'movement']=0

train_data['movement'] = train_data['movement'].shift(-1)
test_data['movement'] = test_data['movement'].shift(-1)

train_data['Date'] = train_data['Date'].map(lambda x:time.mktime(datetime.datetime.strptime(x, "%d-%b-%Y").timetuple()))
test_data['Date'] = test_data['Date'].map(lambda x:time.mktime(datetime.datetime.strptime(x, "%d-%b-%Y").timetuple()))
test_data.head(5)

,Date,Open Price,Close Price,movement
0,1.514822e+09,2683.73,2695.81,1.0
1,1.514909e+09,2697.85,2713.06,1.0
2,1.514995e+09,2719.31,2723.99,1.0
3,1.515082e+09,2731.33,2743.15,1.0
4,1.515341e+09,2742.67,2747.71,1.0


- Drop 掉平移後的NaN
- 把平移會導致的型別改變改回來
- 切割數據

In [58]:
train_data = train_data.dropna()
test_data = test_data.dropna()

train_data['movement'] = train_data['movement'].astype('int_')
test_data['movement'] = test_data['movement'].astype('int_')

x_train = train_data.loc[:, train_data.columns != 'movement']
y_train = train_data[['movement']]
x_test = test_data.loc[:, test_data.columns != 'movement']
y_test = test_data[['movement']]
x_train.head(5)

,Date,Open Price,Close Price
0,1.230826e+09,902.99,931.80
1,1.231085e+09,929.17,927.45
2,1.231171e+09,931.17,934.70
3,1.231258e+09,927.45,906.65
4,1.231344e+09,905.73,909.73


In [59]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression().fit(x_train, y_train)

train_preds = model.predict(x_train)
test_preds = model.predict(x_test)
train_acc = metrics.accuracy_score(y_train, train_preds)
test_acc = metrics.accuracy_score(y_test, test_preds)
print('Train accuracy: %s' % train_acc)
print('Test accuracy: %s' % test_acc)

Train accuracy: 0.5461776403004861
Test accuracy: 0.50199203187251


C:\Users\Hung-Chih\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Hung-Chih\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
